In [1]:
import sys
sys.path.append("..")
import msgspec
import faker
from tqdm.auto import tqdm
from typing import Any, cast
import pickle
import gc


In [2]:
fake = faker.Faker("ru_RU")

In [3]:
from dkvs.bptree import BPTree
BPTree.HALF_INODE_SIZE = 512
BPTree.MAX_INODE_SIZE = BPTree.HALF_INODE_SIZE * 2
BPTree.HALF_LEAF_SIZE = 1024
BPTree.MAX_LEAF_SIZE = BPTree.HALF_LEAF_SIZE * 2

In [4]:
class MyKey(msgspec.Struct, frozen=True, order=True):
    full_name: str

class MyValue(msgspec.Struct):
    data: Any

In [5]:
index = BPTree[MyKey, MyValue]()

In [6]:
faker.Faker.seed(1)
fake = faker.Faker("ru_RU")

In [7]:
data = []
gc.collect()

340

In [8]:
%%time
for i in tqdm(range(10_000_000)):
    val = fake.name() + " - " + str(i)
    resp = index.insert(MyKey(val), fake.address())
    if (resp[0]) is None:
        print(f"Duplicate key: {val}: {resp[1]}")
    data.append(val)

  0%|          | 0/10000000 [00:00<?, ?it/s]

CPU times: user 19min 24s, sys: 11.9 s, total: 19min 36s
Wall time: 19min 32s


In [9]:
%%time
index.validate(), gc.collect()

CPU times: user 8.37 s, sys: 0 ns, total: 8.37 s
Wall time: 8.37 s


({}, 18)

In [10]:
len(data), len(data) / (60 * 19 + 32)

(10000000, 8532.423208191127)

In [11]:
index.print_node(index.tree[index.root_node])


node-idx:1028, prev:None, next:None, level: 2  <============ *ROOT*
      |-> MyKey(full_name='Валентина Мироновна Виноградова - 3558'): 1, 4103
      |-> MyKey(full_name='Евгений Тимурович Гусев - 2651'): 4103, 2053
      |-> MyKey(full_name='Калашникова Вера Тимуровна - 704'): 2053, 4105
      |-> MyKey(full_name='Любовь Степановна Носова - 1499'): 4105, 1027
      |-> MyKey(full_name='Никонова Прасковья Ильинична - 1771'): 1027, 4371
      |-> MyKey(full_name='Савельев Фирс Гертрудович - 2924'): 4371, 2087
      |-> MyKey(full_name='Ульяна Болеславовна Кулакова - 3972'): 2087, 4147


In [12]:
index.print_node(index.tree[0])


node-idx:0, prev:None, next:4476, level: 0 
|-> MyKey(full_name='Абрамов Август Ааронович - 3724287')
|-> MyKey(full_name='Абрамов Август Адрианович - 4056796')
|-> MyKey(full_name='Абрамов Август Андреевич - 8644668')
|-> MyKey(full_name='Абрамов Август Ануфриевич - 3095859')
|-> MyKey(full_name='Абрамов Август Ануфриевич - 4306850')
|-> MyKey(full_name='Абрамов Август Артурович - 133017')
|-> MyKey(full_name='Абрамов Август Бориславович - 4348766')
|-> MyKey(full_name='Абрамов Август Борисович - 6598568')
|-> MyKey(full_name='Абрамов Август Валерьянович - 8571816')
|-> MyKey(full_name='Абрамов Август Васильевич - 511824')
|-> MyKey(full_name='Абрамов Август Всеволодович - 4197942')
|-> MyKey(full_name='Абрамов Август Всеволодович - 4602647')
|-> MyKey(full_name='Абрамов Август Гаврилович - 9755808')
|-> MyKey(full_name='Абрамов Август Гертрудович - 4338570')
|-> MyKey(full_name='Абрамов Август Гертрудович - 7900977')
|-> MyKey(full_name='Абрамов Август Гордеевич - 6504747')
|-> MyKe

In [13]:
index.print_node(index.tree[1])


node-idx:1, prev:None, next:4103, level: 1 
   |-> MyKey(full_name='Абрамов Бронислав Георгиевич - 5960901'): 0, 4476
   |-> MyKey(full_name='Абрамов Данила Чеславович - 1266626'): 4476, 2185
   |-> MyKey(full_name='Абрамов Каллистрат Валерьянович - 5680066'): 2185, 4328
   |-> MyKey(full_name='Абрамов Милий Геннадиевич - 228671'): 4328, 1074
   |-> MyKey(full_name='Абрамов Петр Андреевич - 357332'): 1074, 5345
   |-> MyKey(full_name='Абрамов Соломон Измаилович - 2234824'): 5345, 2406
   |-> MyKey(full_name='Абрамов Эрнест Юлианович - 668839'): 2406, 4705
   |-> MyKey(full_name='Абрамова Ангелина Львовна - 896214'): 4705, 519
   |-> MyKey(full_name='Абрамова Галина Вячеславовна - 554516'): 519, 4345
   |-> MyKey(full_name='Абрамова Зинаида Никифоровна - 2776207'): 4345, 2116
   |-> MyKey(full_name='Абрамова Лора Наумовна - 631344'): 2116, 4248
   |-> MyKey(full_name='Абрамова Наина Ниловна - 1637820'): 4248, 1042
   |-> MyKey(full_name='Абрамова Раиса Ивановна - 32122'): 1042, 4107
  

In [14]:
index.print_node(index.tree[0])


node-idx:0, prev:None, next:4476, level: 0 
|-> MyKey(full_name='Абрамов Август Ааронович - 3724287')
|-> MyKey(full_name='Абрамов Август Адрианович - 4056796')
|-> MyKey(full_name='Абрамов Август Андреевич - 8644668')
|-> MyKey(full_name='Абрамов Август Ануфриевич - 3095859')
|-> MyKey(full_name='Абрамов Август Ануфриевич - 4306850')
|-> MyKey(full_name='Абрамов Август Артурович - 133017')
|-> MyKey(full_name='Абрамов Август Бориславович - 4348766')
|-> MyKey(full_name='Абрамов Август Борисович - 6598568')
|-> MyKey(full_name='Абрамов Август Валерьянович - 8571816')
|-> MyKey(full_name='Абрамов Август Васильевич - 511824')
|-> MyKey(full_name='Абрамов Август Всеволодович - 4197942')
|-> MyKey(full_name='Абрамов Август Всеволодович - 4602647')
|-> MyKey(full_name='Абрамов Август Гаврилович - 9755808')
|-> MyKey(full_name='Абрамов Август Гертрудович - 4338570')
|-> MyKey(full_name='Абрамов Август Гертрудович - 7900977')
|-> MyKey(full_name='Абрамов Август Гордеевич - 6504747')
|-> MyKe

In [15]:
index.min()[1]

MyKey(full_name='Абрамов Август Ааронович - 3724287')

In [16]:
index.print_node(index.max()[0])


node-idx:4860, prev:2473, next:None, level: 0 
|-> MyKey(full_name='тов. Щукина Ангелина Викторовна - 614516')
|-> MyKey(full_name='тов. Щукина Ангелина Владиславовна - 854425')
|-> MyKey(full_name='тов. Щукина Ангелина Леонидовна - 8211881')
|-> MyKey(full_name='тов. Щукина Ангелина Михайловна - 3372368')
|-> MyKey(full_name='тов. Щукина Ангелина Никифоровна - 6160431')
|-> MyKey(full_name='тов. Щукина Анжела Валентиновна - 5868708')
|-> MyKey(full_name='тов. Щукина Анжела Леоновна - 4347619')
|-> MyKey(full_name='тов. Щукина Анжела Наумовна - 2554818')
|-> MyKey(full_name='тов. Щукина Анжелика Георгиевна - 4057181')
|-> MyKey(full_name='тов. Щукина Анжелика Григорьевна - 2049668')
|-> MyKey(full_name='тов. Щукина Анжелика Мироновна - 7261662')
|-> MyKey(full_name='тов. Щукина Анжелика Николаевна - 9230379')
|-> MyKey(full_name='тов. Щукина Анжелика Романовна - 6372972')
|-> MyKey(full_name='тов. Щукина Анна Натановна - 4704498')
|-> MyKey(full_name='тов. Щукина Антонина Дмитриевна -

In [17]:
index.max()[1]

MyKey(full_name='тов. Якушева Юлия Эльдаровна - 4111427')

In [18]:
%%time
i = 0
for i, s in enumerate(index.values):
    if s is not None:
        i += 1


CPU times: user 1.34 s, sys: 9.79 ms, total: 1.35 s
Wall time: 1.35 s


In [19]:
len(index.values), len(index.keys)

(10000000, 10000000)

In [20]:
index.values[:10]

['ст. Североморск, бул. Воровского, д. 81 к. 166, 439150',
 'с. Арсеньев, алл. Бабушкина, д. 9/4 стр. 4, 740681',
 'д. Калачинск, наб. Тюленина, д. 1/8 к. 76, 585178',
 'клх Артем, ул. Свердлова, д. 3/3 к. 40, 685957',
 'ст. Тобольск, пер. Высокий, д. 1/8, 983867',
 'д. Армавир, бул. Тепличный, д. 91, 011070',
 'к. Любань, алл. Большая, д. 9, 447577',
 'к. Вуктыл, бул. Санаторный, д. 1/4 к. 162, 786838',
 'г. Макушино, алл. Коммунистическая, д. 2, 442694',
 'ст. Юрьевец (Иван.), ул. Циолковского, д. 25 стр. 79, 716487']

In [21]:
index2 = BPTree[MyKey, MyValue]()
gc.collect()

0

In [22]:
%%time
for k, v in tqdm(zip(index.keys, index.values)):
    resp = index2.insert(k, v)
    if resp[0] is None:
        print("Duplicated:", k, v)


0it [00:00, ?it/s]

CPU times: user 5min 40s, sys: 2.07 s, total: 5min 42s
Wall time: 5min 41s


In [23]:
10000000 / (5 * 60 + 41)

29325.513196480937